In [54]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/MyDrive/MyRepos/ParseCDL/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
import pyparsing as _p
import netlist as nl

In [146]:
netlist = """
***************************************
* auCdl Netlist
*
* Library Name: ML_DATA_LIB
* View Name : asdsa
* Top Cewll Name : sadasdsdf
* Netlisted On: FGerv, asdfasfd, 2 10:35:01 2022
***************************************
.INCLUDE $calibre_source_added_place
*.BIPOLAR
*.REST = 2000
*.MEGA
.PARAM

*.EQUIV xcvr_avdd_clk = n12
*+    loaddely = n13
*+    rst_n = n14
.SUBCKT BUFFER A Z gnd gnds vdd vdds
+ n1 n2 n3
+ n4 n5
* net net0 = nnnn
* net Z = /zzzz/asdasd00
M1 Z net0 vdd vdds mypmos w=5 l=1
* net A = /gggg/asdsa
M2 net0 A vdd vdds mypmos w=5 l=1
+ area=122 fin=34
+ sde=44
M3 Z net0 gnd gnds mynmos w=2 l=1
M4 net0 A gnd gnds mynmos w=2 l=1
X10157 n42 n42 n42 moscap_rf_nw m=1.0 br=4.0 gr=4.0 lr=135e-9 nfin=8.0
R10158 n5332 n5331 100e-3 $[SH]
.ENDS

I1 net0 net1 0 gnds! vdd! vdds! BUFFER
I2 net1 net2 0 gnds! vdd! vdds! INVERTER
"""

In [160]:
def handle_instance(token):
    inst = token.instance
    name = inst[0]
    pins, reference, params = divide_body(inst.instbody)
    parameters = dict_params(params)
    if not parameters and reference == '$[SH]':
      # this is the resistance
      reference = name + "_res"
      parameters['res'] = pins.pop()
    pins = list(map(lambda x: net_dict.get(x, x), pins))
    i = nl.instance(name, pins, reference, parameters)
    return [i]

In [161]:
def divide_body(objs):
    out = len(objs)
    try:
        out = next(idx for idx, obj in enumerate(objs) if "=" in obj)
    except StopIteration:
        pass
    return objs[:out-1], objs[out-1], objs[out:]

In [162]:
def dict_params(params):
    m_split = list(map(lambda x: x.split("="), params))
    key = map(lambda x: x[0], m_split)
    value = map(lambda x:x[1], m_split)
    return dict(zip(key, value))

In [163]:
def handle_ext_nets(token):
    inst = token.ext_nets
    net_dict[inst.key] = inst.value
    return []
    # return [{inst.key : inst.value}]

In [164]:
def handle_int_nets(token):
    inst = token.int_nets
    net_dict[inst.key] = inst.value 
    return []
    # return [{inst.key : inst.value}]

In [165]:
def handle_subcircuit(token):
    sc = token.subcircuit
    nets = sc.nets
    nets = list(map(lambda x: net_dict.get(x, x), nets))
    name = sc.name
    subcontent = sc.subnetlist
    instances = list(filter(lambda x: type(x) == nl.instance, subcontent))
    s = nl.subcircuit(name, nets, instances)
    return [s]

In [166]:
def parse_spectre(netlist_string):
    # newlines are part of the grammar, thus redifine the whitespaces without it
    ws = ' \t'
    _p.ParserElement.setDefaultWhitespaceChars(ws)

    # spectre netlist grammar definition
    EOL = _p.LineEnd().suppress() # end of line
    linebreak = _p.Suppress(_p.LineEnd() + '+') # breaking a line with backslash newline
    identifier=_p.Word(_p.alphanums+'_!<>/') # a name for...
    number=_p.Word(_p.nums + ".") # a number
    expression = _p.Word(_p.alphanums+'._*+-/()=!<>$[]')
    net = expression # a net
    nets = _p.Group(_p.OneOrMore(net | linebreak)) # many nets
    cktname = identifier # name of a subcircuit
    cktname_end = _p.Keyword(".ENDS").suppress()
    instname = identifier
    instance = _p.Group(instname('instname') + nets('instbody') + EOL).setResultsName('instance')
    int_net = _p.Suppress("* net") + identifier('key') + _p.Suppress("=") + identifier('value') + EOL
    int_nets = _p.Group(int_net).setResultsName('int_nets')
    subcircuit_content = _p.Group(_p.ZeroOrMore(instance | int_nets | EOL)).setResultsName('subnetlist')
    cir_net = identifier
    cir_nets = _p.Group(_p.OneOrMore(cir_net('net') | linebreak))
    ext_net = _p.Suppress(_p.Literal("*.EQUIV") | _p.Literal("*+"))  + identifier('value') + _p.Suppress("=") + identifier('key') + EOL
    ext_nets = _p.Group(ext_net).setResultsName('ext_nets')
    subcircuit = _p.Group(
        # matches subckt <name> <nets> <newline>
        _p.Keyword(".SUBCKT").suppress() + cktname('name') + cir_nets('nets') + EOL  
        # matches the content of the subcircuit
        + subcircuit_content
        # matches ends <name> <newline>
        + cktname_end + EOL).setResultsName('subcircuit')
    include = _p.Group(_p.Keyword(".INCLUDE") + ... + _p.Keyword(".PARAM"))
    star_line = _p.Group(_p.Combine(_p.Keyword("***") + ... + _p.Keyword("***")))
    netlist_element = subcircuit | instance | _p.Suppress(include) | _p.Suppress(star_line) | ext_nets | EOL
    netlist = _p.ZeroOrMore(netlist_element) + _p.StringEnd()
    
    global net_dict
    net_dict = {}
    ext_nets.setParseAction(handle_ext_nets)
    int_nets.setParseAction(handle_int_nets)
    instance.setParseAction(handle_instance)
    subcircuit.setParseAction(handle_subcircuit)
    
    return netlist.parseString(netlist_string);

In [167]:
result1 = parse_spectre(netlist)

In [168]:
net_dict

{'A': '/gggg/asdsa',
 'Z': '/zzzz/asdasd00',
 'n12': 'xcvr_avdd_clk',
 'n13': 'loaddely',
 'n14': 'rst_n',
 'net0': 'nnnn'}

In [170]:
result1

ParseResults([BUFFER, <netlist.instance object at 0x7f08a9d50d50>, <netlist.instance object at 0x7f08a9c69d10>], {'subcircuit': [BUFFER], 'instance': [<netlist.instance object at 0x7f08a9c69d10>]})

In [192]:
# for ele in result1:
#     print(ele.get_name())
#     print(ele)
#     if ele.get_name() == 'subcircuit':
#         for _ in ele:
#             if type(_) != str:
#                 print(_.get_name())
#                 print(_)
#                 if _.get_name() == "subnetlist":
#                     for t in _:
#                         # print(t.get_name())
#                         print(t)
#     print('\n')
for ele in result1:
  print(ele)

subcircuit BUFFER({'/gggg/asdsa': BUFFER./gggg/asdsa, '/zzzz/asdasd00': BUFFER./zzzz/asdasd00, 'gnd': BUFFER.gnd, 'gnds': BUFFER.gnds, 'vdd': BUFFER.vdd, 'vdds': BUFFER.vdds, 'n1': BUFFER.n1, 'n2': BUFFER.n2, 'n3': BUFFER.n3, 'n4': BUFFER.n4, 'n5': BUFFER.n5, 'nnnn': BUFFER.nnnn, 'n42': BUFFER.n42, 'n5332': BUFFER.n5332, 'n5331': BUFFER.n5331}):{'M1': 'mypmos', 'M2': 'mypmos', 'M3': 'mynmos', 'M4': 'mynmos', 'X10157': 'moscap_rf_nw', 'R10158': 'R10158_res'}
instance I1@BUFFER{}
instance I2@INVERTER{}


In [194]:
for ele in result1:
  if ele.typeof == 'subcircuit':
    for instance in ele.instances:
      print(instance)

instance M1@mypmos{'w': '5', 'l': '1'}
instance M2@mypmos{'w': '5', 'l': '1', 'area': '122', 'fin': '34', 'sde': '44'}
instance M3@mynmos{'w': '2', 'l': '1'}
instance M4@mynmos{'w': '2', 'l': '1'}
instance X10157@moscap_rf_nw{'m': '1.0', 'br': '4.0', 'gr': '4.0', 'lr': '135e-9', 'nfin': '8.0'}
instance R10158@R10158_res{'res': '100e-3'}
